In [17]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [18]:
reviews = pd.read_csv('C:/JK/Masters Studies/Spring 2018/Information Retrieval/Project/data/reviews.csv/reviews.csv')
# reviews['listing_id'].value_counts()
# vc = reviews['listing_id'].value_counts()

In [19]:
def remove_nonalphanumeric(text):
    return re.sub("[^a-zA-Z0-9]"," ", str(text)).lower()

In [20]:
unique_listings = reviews.listing_id.unique()
len(unique_listings)

3986

In [21]:
reviews=reviews.groupby('listing_id').filter(lambda g: (g.listing_id.size >= 100))
# reviews = reviews[reviews['listing_id'] != 15041925]
# reviews = reviews[reviews['listing_id'] !=19102778]

unique_listings = reviews.listing_id.unique()
len(unique_listings)

290

In [22]:
reviews['comments_1'] = reviews['comments'].apply(remove_nonalphanumeric)
reviews['tokenized_comments'] = reviews['comments_1'].apply(word_tokenize)
stopset = stopwords.words('english') + list(string.punctuation)
reviews['stop_comments'] = reviews['tokenized_comments'].apply(lambda x: [item for item in x if item not in stopset])
def func(row):
    return " ".join(row)
reviews['data'] = reviews['stop_comments'].apply(lambda x: func(x))

In [23]:
def display_topics(H,W,feature_names,documents,documents_id,listing_id,words,no_top_words,no_top_documents):
    result_list = []
    for topic_idx, topic in enumerate(H):
        message = " ".join(([feature_names[i]
                             for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            result = (message,documents_id[doc_index],listing_id[doc_index],documents[doc_index],words[doc_index])
            result_list.append(result)

    return result_list

In [ ]:
no_top_words = 3
no_top_documents = 5
result=[]

In [ ]:
for each_listing in unique_listings:
#     print(each_listing)
    
    listing_reviews=reviews[reviews['listing_id']==each_listing]
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000000, ngram_range = (1,2))
    tf = tf_vectorizer.fit_transform(listing_reviews['data'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    lda = LatentDirichletAllocation(n_topics = 5, max_iter = 5,learning_method='online',learning_offset=50., random_state=0).fit(tf)
    lda_W = lda.transform(tf)
    lda_H = lda.components_
    output = display_topics(lda_H,lda_W,tf_feature_names,list(listing_reviews['comments']),list(listing_reviews['id']),list(listing_reviews['listing_id']),list(listing_reviews['data']),no_top_words,no_top_documents)
    result += output

result_df = pd.DataFrame(result, columns=['Topic','Review_Id','Listing_Id','Documents','Words'])
result_df.to_csv("Result_entire1.csv",columns=["Topic","Review_Id","Listing_Id","Documents","Words"])